# 0.1. Импортируем зависимости

In [209]:
import itertools
import pandas as pd
import psycopg2
from psycopg2.extensions import quote_ident
import re
import collections

In [210]:
def print_columns(cursor):
    """Выводит список наименований колонок результата выполнения запроса"""
    print([desc[0] for desc in cursor.description])

# 0.2. Подключимся к базе
Предполагается, что соответствующая база и пользователь с соответствующими правами уже созданы, таблиц с наименованиями, конфликтующих с наименованиями таблиц задания в базе нет, подключение таким образом возможно и т.п.

In [211]:
DATABASE = 'db_hw_2'
USER = 'db_hw_2_user'

connection = psycopg2.connect(dbname=DATABASE, user=USER)
connection.autocommit = True
cursor = connection.cursor()

# 1.1. Создадим таблицы с соответствующими структурами

In [212]:
cursor.execute("""
    CREATE TABLE customer_20240101 (
        customer_id int4,
        first_name varchar(50),
        last_name varchar(50),
        gender varchar(30),
        dob varchar(50),
        job_title varchar(50),
        job_industry_category varchar(50),
        wealth_segment varchar(50),
        deceased_indicator varchar(50),
        owns_car varchar(30),
        address varchar(50),
        postcode varchar(30),
        state varchar(30),
        country varchar(30),
        property_valuation int4
    )
""")

cursor.execute("""
    CREATE TABLE transaction_20240101 (
        transaction_id int4,
        product_id int4,
        customer_id int4,
        transaction_date varchar(30),
        online_order varchar(30),
        order_status varchar(30),
        brand varchar(30),
        product_line varchar(30),
        product_class varchar(30),
        product_size varchar(30),
        list_price float4,
        standard_cost float4
    )
""")

# 1.2. Загрузим данные в базу из csv-файлов 
Для этого, после чтения данных из каждого файла, будем формировать и выполнять SQL-запрос `INSERT INTO ...` вида `INSERT INTO table_name ( column_name [, ...] ) VALUES (expression, ...), ...` согласно [официальной документации PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html).

Заодно выведем наименования колонок таблиц.

In [213]:
for filename in ['customer.csv', 'transaction.csv']:
    # прочитаем данные из соответствующего файла
    # NULL-ами (в терминах SQL) будем считать пустые строки и строку "null" в любом регистре
    # типы используем такие, которые позволят избежать нежелательного изменения данных
    # с учётом типов, заданных выше для соответствующих колонок наших таблиц в базе
    df = pd.read_csv(filename, nrows=1, sep=';')
    dtypes = {k: str for k in df.columns}
    dtypes.update(list_price=float, standard_cost=float)
    df = pd.read_csv(
        filename, 
        sep=';',
        na_values=('', ),
        keep_default_na=False,
        decimal=',',
        dtype=dtypes,
    )
    table = f'{filename[:-4]}_20240101'
    df = df.map(lambda x: None if isinstance(x, str) and re.match('^null$', x, flags=re.I) else x)
    df = df.where(pd.notnull(df), None)
    
    # значения для вставки и наименования колонок
    values = list(itertools.chain.from_iterable(df.values.tolist()))
    cols = [i.lower() for i in df.columns]
    del df
    
    # Сформируем (и выполним) запрос INSERT INTO следующего вида
    # INSERT INTO table_name ( column_name [, ...] ) VALUES (expression, ...), ...
    # согласно официальной документации https://www.postgresql.org/docs/current/sql-insert.html
    cols_sql = ', '.join(quote_ident(i, cursor) for i in cols)
    print(f'Колонки таблицы "{table}": {cols_sql}')
    values_sql = ','.join([f'(' + ','.join(['%s'] * len(cols)) + ')'] * (len(values) // len(cols)))
    table_ident = quote_ident(table, cursor)
    cursor.execute(f'INSERT INTO {table_ident} ({cols_sql}) VALUES {values_sql}', values)

Колонки таблицы "customer_20240101": "customer_id", "first_name", "last_name", "gender", "dob", "job_title", "job_industry_category", "wealth_segment", "deceased_indicator", "owns_car", "address", "postcode", "state", "country", "property_valuation"
Колонки таблицы "transaction_20240101": "transaction_id", "product_id", "customer_id", "transaction_date", "online_order", "order_status", "brand", "product_line", "product_class", "product_size", "list_price", "standard_cost"


## Получим полезную информацию о данных для использования в дальнейшем
По наименованиям колонок и в результате осмотра данных в файлах / таблицах видим, что потенциальные ключи тут скорее всего `customer_id` для таблицы `customer_20240101` и `transaction_id` для таблицы `transaction_20240101`, а объединять таблицы скорее всего разумно по колонке `customer_id`, получим дополнительные данные, чтобы подкрепить данные предположения.

In [214]:
cursor.execute("""
    SELECT count(DISTINCT customer_id) = count(*), bool_or(customer_id IS NULL)
    FROM customer_20240101
""")
print(('В таблице customer_20240101 значения customer_id уникальны: %s,'
       + ' есть ли пропуски (null) в customer_id: %s') % cursor.fetchall()[0])
cursor.execute("""
    SELECT count(DISTINCT transaction_id) = count(*), bool_or(transaction_id IS NULL)
    FROM transaction_20240101
""")
print(('В таблице transaction_20240101 значения transaction_id уникальны: %s,'
       ' есть ли пропуски (null) в transaction_id: %s') % cursor.fetchall()[0])
cursor.execute("""
    SELECT count(*), sum((c.customer_id IS NULL)::int)
    FROM transaction_20240101 t LEFT JOIN customer_20240101 c USING (customer_id)
""")
print(('В таблице transaction_20240101 количество записей (строк): %s, записей,'
       ' по которым нет соответствующей записи в customer_20240101,'
       ' если присоединять по равенству customer_id: %s') % cursor.fetchall()[0])

В таблице customer_20240101 значения customer_id уникальны: True, есть ли пропуски (null) в customer_id: False
В таблице transaction_20240101 значения transaction_id уникальны: True, есть ли пропуски (null) в transaction_id: False
В таблице transaction_20240101 количество записей (строк): 20000, записей, по которым нет соответствующей записи в customer_20240101, если присоединять по равенству customer_id: 3


Из вышеполученных результатов видим, что атрибуты `customer_id` и `transaction_id` являются потенциальными ключами таблиц `customer_20240101` и `transaction_20240101` соответственно.
Для надёжности, ниже создадим соответствующие ограничения "первичный ключ", что не получится, если вывод неверен.
Кроме того, видно, что почти всем значениям `customer_id` в таблице `transaction_20240101` соответствует значение `customer_id` в таблице `customer_20240101`, поэтому будем соединять их по равенству значений этой колонки, когда для запроса нам будут нужны данные из обеих таблиц.
Клиентами везде ниже будем считать только указанных в таблице `customer_20240101` клиентов.

In [215]:
cursor.execute('ALTER TABLE customer_20240101 ADD PRIMARY KEY (customer_id)')
cursor.execute('ALTER TABLE transaction_20240101 ADD PRIMARY KEY (transaction_id)')

Получилось наложить соответствующие ограничения "первичный ключ", чем дополнительно подтвердилось, что можем полагаться на то, что ключам customer_id, transaction_id соответствуют уникальные строки в соответствующих таблицах, где они являются первичными ключами, пропусков (null в терминах SQL) в этих полях нет.

Поскольку ниже нам понадобится знать, является ли заказ онлайн-заказом или нет, посмотрим, какие значения бывают в соответствующей колонке.

In [216]:
cursor.execute('SELECT DISTINCT online_order FROM transaction_20240101')
cursor.fetchall()

[(None,), ('True',), ('False',)]

Видим, что для получения онлайн-заказов разумно будет использовать условие `online_order = 'True'`.

Далее посмотрим, какие значения бывают у статуса подтверждённости транзакции.

In [217]:
cursor.execute('SELECT DISTINCT order_status FROM transaction_20240101')
cursor.fetchall()

[('Approved',), ('Cancelled',)]

Видим, что подтверждённые транзакции разумно будет выбирать по значению `order_status = 'Approved'`

# 2. Выполним запросы для вывода разной информации

Результат каждого запроса будем выводить в виде списка кортежей, который возвращает метод `cursor.fetchall` соответствующего объекта из библиотеки `psycopg2`

Ниже в запросах будем использовать оператор `DISTINCT` для обеспечения вывода уникальных строк без дублей, функцию `to_date` для приведения строковых типов к типу `date`.

## 2.1. Выведем распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [218]:
cursor.execute("""
    SELECT job_industry_category, count(*) AS number_of_clients
    FROM customer_20240101
    GROUP BY job_industry_category
    ORDER BY number_of_clients DESC
""")
cursor.fetchall()

[('Manufacturing', 799),
 ('Financial Services', 774),
 ('n/a', 656),
 ('Health', 602),
 ('Retail', 358),
 ('Property', 267),
 ('IT', 223),
 ('Entertainment', 136),
 ('Argiculture', 113),
 ('Telecommunications', 72)]

## 2.2. Найдём сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.
Будем считать, что совпадающие по названию месяцы разных лет - это разные месяцы. Выводить их будем в формате YYYY-MM.

In [270]:
cursor.execute("""
    SELECT to_char(to_date(transaction_date, 'DD.MM.YYYY'), 'YYYY-MM') AS month,
           c.job_industry_category,
           sum(t.list_price) AS transactions_sum
    FROM transaction_20240101 t
    INNER JOIN customer_20240101 c
        USING (customer_id)
    GROUP BY month, c.job_industry_category
    ORDER BY month, c.job_industry_category
""")
cursor.fetchall()

[('2017-01', 'Argiculture', 43513.812),
 ('2017-01', 'Entertainment', 64089.934),
 ('2017-01', 'Financial Services', 366383.78),
 ('2017-01', 'Health', 286860.44),
 ('2017-01', 'IT', 107783.414),
 ('2017-01', 'Manufacturing', 365232.38),
 ('2017-01', 'Property', 100686.97),
 ('2017-01', 'Retail', 182375.72),
 ('2017-01', 'Telecommunications', 31210.2),
 ('2017-01', 'n/a', 316819.72),
 ('2017-02', 'Argiculture', 60016.81),
 ('2017-02', 'Entertainment', 63965.99),
 ('2017-02', 'Financial Services', 375961.8),
 ('2017-02', 'Health', 268525.75),
 ('2017-02', 'IT', 93961.79),
 ('2017-02', 'Manufacturing', 389260.53),
 ('2017-02', 'Property', 112255.41),
 ('2017-02', 'Retail', 146107.58),
 ('2017-02', 'Telecommunications', 27678.2),
 ('2017-02', 'n/a', 260711.9),
 ('2017-03', 'Argiculture', 49048.492),
 ('2017-03', 'Entertainment', 77122.44),
 ('2017-03', 'Financial Services', 322268.28),
 ('2017-03', 'Health', 290575.88),
 ('2017-03', 'IT', 100212.4),
 ('2017-03', 'Manufacturing', 377784.72

## 2.3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.
NULL брэндом считать не будем, но если нужно было бы, для этого тут достаточно было бы убрать `AND t.brand IS NOT NULL`.

In [220]:
cursor.execute("""
    SELECT t.brand, count(*) AS num_of_approved_online_transactions
    FROM transaction_20240101 t
    INNER JOIN customer_20240101 c
        USING (customer_id)
    WHERE c.job_industry_category = 'IT'
        AND t.online_order = 'True'
        AND t.order_status = 'Approved'
        AND t.brand IS NOT NULL
    GROUP BY t.brand
""")
cursor.fetchall()

[('Giant Bicycles', 89),
 ('Norco Bicycles', 92),
 ('OHM Cycles', 78),
 ('Solex', 101),
 ('Trek Bicycles', 82),
 ('WeareA2B', 90)]

## 2.4. Найдём по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества транзакций. Выполним двумя способами: используя только group by и используя только оконные функции. Сравним результат.
Наименования колонок результата тоже выведем для удобства тут в ноутбуке, чтобы было легче ориентироваться в результатах выполнения запроса.

### 2.4.1. Способ 1: используя GROUP BY без оконных функций
Для вычисления суммы дополнительно приведём тип list_price к numeric, чтобы повысить точность по сравнению с исходным float4.

In [242]:
cursor.execute("""
    SELECT c.customer_id,
           c.first_name,
           c.last_name,
           sum(t.list_price) AS list_price_sum,
           max(t.list_price) AS max_list_price,
           min(t.list_price) AS min_list_price,
           count(*) AS transactions_count
    FROM transaction_20240101 t
    INNER JOIN customer_20240101 c
        USING (customer_id)
    GROUP BY c.customer_id, c.first_name, c.last_name
    ORDER BY list_price_sum DESC, transactions_count DESC
""")
print_columns(cursor)
group_by_rows = cursor.fetchall()
group_by_rows

['customer_id', 'first_name', 'last_name', 'list_price_sum', 'max_list_price', 'min_list_price', 'transactions_count']


[(2183, 'Jillie', 'Fyndon', 19071.322, 2005.66, 230.91, 14),
 (1129, 'Hercule', None, 18349.27, 1992.93, 290.62, 13),
 (1597, 'Jeffry', 'Slowly', 18052.68, 2091.47, 360.4, 12),
 (941, 'Tye', 'Doohan', 17898.459, 2091.47, 1057.51, 10),
 (2788, 'Melantha', 'Pickburn', 17258.94, 2083.94, 183.86, 11),
 (936, 'Raffaello', 'Godleman', 17160.24, 2005.66, 183.86, 12),
 (1887, 'Kynthia', 'Purcer', 17133.932, 2091.47, 688.63, 11),
 (1302, 'Ericka', 'Eggers', 17035.83, 1977.36, 71.16, 13),
 (1140, 'Deana', 'Rathbourne', 16199.24, 2083.94, 183.86, 13),
 (2309, 'Herc', 'McIlhone', 16122.341, 2091.47, 290.62, 12),
 (729, 'Ammamaria', 'Standbridge', 15825.999, 2091.47, 586.45, 10),
 (1103, 'Glynnis', 'Sailor', 15447.92, 1977.36, 230.91, 12),
 (1317, 'Barrett', 'Lindley', 15370.81, 2091.47, 569.56, 9),
 (2874, 'Delores', 'Ashcroft', 15091.91, 2005.66, 544.05, 11),
 (2762, 'Rozamond', 'Franceschino', 15071.26, 1997.68, 183.86, 10),
 (255, 'Keeley', 'Kruger', 14949.91, 1810.0, 441.49, 12),
 (1672, 'Shar

### 2.4.2. Способ 2: используя оконные функции без GROUP BY

In [258]:
cursor.execute("""
    SELECT DISTINCT 
        c.customer_id,
        c.first_name,
        c.last_name,
        sum(t.list_price) OVER customers_window AS list_price_sum,
        max(t.list_price) OVER customers_window AS max_list_price,
        min(t.list_price) OVER customers_window AS min_list_price,
        count(*) OVER customers_window AS transactions_count
    FROM transaction_20240101 t
    INNER JOIN customer_20240101 c
        USING (customer_id)
    WINDOW customers_window AS (PARTITION BY c.customer_id)
    ORDER BY list_price_sum DESC, transactions_count DESC
""")
print_columns(cursor)
window_rows = cursor.fetchall()
window_rows

['customer_id', 'first_name', 'last_name', 'list_price_sum', 'max_list_price', 'min_list_price', 'transactions_count']


[(2183, 'Jillie', 'Fyndon', 19071.32, 2005.66, 230.91, 14),
 (1129, 'Hercule', None, 18349.27, 1992.93, 290.62, 13),
 (1597, 'Jeffry', 'Slowly', 18052.68, 2091.47, 360.4, 12),
 (941, 'Tye', 'Doohan', 17898.459, 2091.47, 1057.51, 10),
 (2788, 'Melantha', 'Pickburn', 17258.94, 2083.94, 183.86, 11),
 (936, 'Raffaello', 'Godleman', 17160.24, 2005.66, 183.86, 12),
 (1887, 'Kynthia', 'Purcer', 17133.93, 2091.47, 688.63, 11),
 (1302, 'Ericka', 'Eggers', 17035.83, 1977.36, 71.16, 13),
 (1140, 'Deana', 'Rathbourne', 16199.241, 2083.94, 183.86, 13),
 (2309, 'Herc', 'McIlhone', 16122.34, 2091.47, 290.62, 12),
 (729, 'Ammamaria', 'Standbridge', 15826.0, 2091.47, 586.45, 10),
 (1103, 'Glynnis', 'Sailor', 15447.92, 1977.36, 230.91, 12),
 (1317, 'Barrett', 'Lindley', 15370.81, 2091.47, 569.56, 9),
 (2874, 'Delores', 'Ashcroft', 15091.911, 2005.66, 544.05, 11),
 (2762, 'Rozamond', 'Franceschino', 15071.26, 1997.68, 183.86, 10),
 (255, 'Keeley', 'Kruger', 14949.91, 1810.0, 441.49, 12),
 (1672, 'Sharla'

### 2.4.3. Сравним результаты выполнения запроса, использовавшего group by и запроса, использовавшего оконные функции
Результаты в целом совпадают, всё ОК.
Видны небольшие расхождения, связанные с ограничениями точности вычислений, вносимыми типом данных "float4" у поля list_price (описано в [документации](https://www.postgresql.org/docs/current/datatype-numeric.html#DATATYPE-FLOAT)) и допустимыми различиями результатов сортировки (PostgreSQL не гарантирует, что строки в результатах запроса будут в детерминированном порядке, если порядок не определён в запросе, а в нашем запросе определён порядок только по двум полям, т.е. если две строки совпадают по этим двум полям, то порядок этих строк от запроса к запросу может варьироваться).

## 2.5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишем отдельные запросы для минимальной и максимальной суммы.
Тут будем конвертировать float4 в numeric, чтобы, как рекомендует [документация](https://www.postgresql.org/docs/current/datatype-numeric.html#DATATYPE-FLOAT), получить точные результаты вычислений.
Саму искомую сумму тоже будем выводить (чтобы не выводить, можно просто тут не указывать её в SELECT-е).

### 2.5.1. Сначала напишем запрос для минимальной суммы (имена и фамилии клиентов с минимальной суммой транзакций за весь период)

In [273]:
cursor.execute("""
    WITH customers_transactions AS (
        SELECT c.first_name,
               c.last_name,
               sum(t.list_price::numeric) AS transactions_sum
        FROM customer_20240101 c
        INNER JOIN transaction_20240101 t
            USING (customer_id)
        GROUP BY c.customer_id, c.first_name, c.last_name
    )
    SELECT first_name, last_name, transactions_sum
    FROM customers_transactions
    WHERE transactions_sum = (SELECT min(transactions_sum) FROM customers_transactions)
""")
cursor.fetchall()

[('Hamlen', 'Slograve', Decimal('60.34'))]

### 2.5.2. Теперь напишем запрос для максимальной суммы (имена и фамилии клиентов с максимальной суммой транзакций за весь период)

In [272]:
cursor.execute("""
    WITH customers_transactions AS (
        SELECT c.first_name,
               c.last_name,
               sum(t.list_price::numeric) AS transactions_sum
        FROM customer_20240101 c
        INNER JOIN transaction_20240101 t
            USING (customer_id)
        GROUP BY c.customer_id, c.first_name, c.last_name
    )
    SELECT first_name, last_name, transactions_sum
    FROM customers_transactions
    WHERE transactions_sum = (SELECT max(transactions_sum) FROM customers_transactions)
""")
cursor.fetchall()

[('Jillie', 'Fyndon', Decimal('19071.32'))]

## 2.6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.
Чтобы вывести данные по каждому клиенту используем окно с партицией по клиентам, а чтобы брать данные первой транзакции с помощью функции `first_value`, укажем сортировку по дате транзакции в партициях.

In [267]:
cursor.execute("""
    SELECT DISTINCT
        c.customer_id,
        c.first_name, 
        c.last_name,
        first_value(t.transaction_id) OVER transactions,
        first_value(t.transaction_date) OVER transactions
    FROM customer_20240101 c
    INNER JOIN transaction_20240101 t
        USING (customer_id)
    WINDOW transactions AS (PARTITION BY c.customer_id ORDER BY t.transaction_date)
""")
cursor.fetchall()

[(2780, 'Papageno', 'Beare', 5914, '04.06.2017'),
 (315, 'Lisa', 'Odlin', 11613, '01.10.2017'),
 (574, 'Gerard', 'Steketee', 13305, '12.02.2017'),
 (988, 'Andree', 'Simonato', 1420, '09.09.2017'),
 (3078, 'Mariel', 'Blacksell', 8323, '14.07.2017'),
 (2107, 'Dre', 'Ferrillo', 2211, '10.08.2017'),
 (2544, 'Ellery', 'Bowra', 19913, '04.07.2017'),
 (738, 'Bunnie', 'Whines', 2267, '02.02.2017'),
 (1565, 'Jay', 'Janiszewski', 3816, '21.12.2017'),
 (148, 'Jaquith', 'Maffey', 10310, '11.09.2017'),
 (3260, 'Kelsy', 'Rao', 16517, '07.02.2017'),
 (1547, 'Chiquita', 'Billyard', 5078, '01.09.2017'),
 (1554, 'Land', 'Aarons', 11936, '06.08.2017'),
 (1733, 'Gothart', 'Coomer', 7732, '03.02.2017'),
 (950, 'Bret', 'Ivakhnov', 8573, '13.07.2017'),
 (2733, 'Jordanna', 'Skyme', 18589, '02.12.2017'),
 (2599, 'Pauly', 'Eilles', 9813, '03.07.2017'),
 (1657, 'Stanwood', 'Troillet', 18194, '07.09.2017'),
 (983, 'Shaylyn', 'Riggs', 12800, '05.07.2017'),
 (1690, 'Torey', 'Kensall', 18541, '01.11.2017'),
 (124, '

## 2.7. Выведем имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) 


In [278]:
cursor.execute("""
    -- соединим таблицы и конвертируем даты
    WITH customers_transaction_dates AS (
        SELECT c.customer_id,
               c.first_name,
               c.last_name,
               c.job_title,
               to_date(transaction_date, 'DD.MM.YYYY') AS current_transaction_date
        FROM customer_20240101 c
        INNER JOIN transaction_20240101 t
            USING (customer_id)
    ),
    -- по каждому клиенту посчитаем интервалы между последовательными транзакциями
    customers_transaction_intervals AS (
        SELECT first_name,
               last_name,
               job_title,
               current_transaction_date - lag(current_transaction_date) OVER (
                   PARTITION BY customer_id
                   ORDER BY current_transaction_date
               ) transaction_date_interval
        FROM customers_transaction_dates
    )
    -- выберем только с максимальными интервалами
    SELECT DISTINCT first_name, last_name, job_title
    FROM customers_transaction_intervals
    WHERE transaction_date_interval = (
        SELECT max(transaction_date_interval)
        FROM customers_transaction_intervals
    )
""")
cursor.fetchall()

[('Susanetta', None, 'Legal Assistant')]